**Labolatorium 2**

**Jakub Sacha**, gr. **1**

Łączenie się z bazą danych za pomocą ***psycopg2*** i ***pandas***



In [2]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

**Zadanie 1**

Ile kategorii filmów mamy w wypożyczlani?

In [ ]:
number_of_film_categories = pd.read_sql('select COUNT(DISTINCT category_id) as ile_kategorii_filmow_jest_w_wypozyczalni from film_category', con=connection)
number_of_film_categories

,ile_kategorii_filmow_jest_w_wypożyczalni
0,16



**Zadanie 2**

Wyświetl listę kategorii w kolejności alfabetycznej. 

In [ ]:
film_categories = pd.read_sql('select name as kategorie_filmow_w_wypozyczalni from category ORDER BY name', con=connection)
film_categories

,kategorie_filmow_w_wypozyczalni
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


**Zadanie 3**

Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [ ]:
films = pd.read_sql('select title, release_year from film where release_year in ((select min(release_year) from film), (select max(release_year) from film))',con=connection)
films

,title,release_year
0,Chamber Italian,2006
1,Grosse Wonderful,2006
2,Airport Pollock,2006
3,Bright Encounters,2006
4,Academy Dinosaur,2006
...,...,...
995,Young Language,2006
996,Youth Kick,2006
997,Zhivago Core,2006
998,Zoolander Fiction,2006


Filmy są z 2006 roku, więc kazdy film jest jednocześnie najstarszy i najmłodszy.

**Zadanie 4**

Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [3]:
df = pd.read_sql("select COUNT(*) as ile_bylo_wypozyczen from rental where rental_date >= '2005-07-01 00:00:00' and rental_date <= '2005-08-01 23:59:59'",con=connection)
df

,ile_bylo_wypozyczen
0,7380


**Zadanie 5**

Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?


In [4]:
df = pd.read_sql("select COUNT(*) as ile_bylo_wypozyczen from rental where rental_date >= '2010-01-01 00:00:00' and rental_date <= '2011-02-01 23:59:59'",con=connection)
df

,ile_bylo_wypozyczen
0,0


**Zadanie 6**

Znajdź największą płatność wypożyczenia.

In [5]:
df = pd.read_sql("select * from payment where amount in (select max(amount) from payment)",con=connection)
df

,payment_id,customer_id,staff_id,rental_id,amount,payment_date
0,20403,362,1,14759,11.99,2007-03-21 21:57:24.996577
1,22650,204,2,15415,11.99,2007-03-22 22:17:22.996577
2,23757,116,2,14763,11.99,2007-03-21 22:02:26.996577
3,24553,195,2,16040,11.99,2007-03-23 20:47:59.996577
4,24866,237,2,11479,11.99,2007-03-02 20:46:39.996577
5,28799,591,2,4383,11.99,2007-04-07 19:14:17.996577
6,28814,592,1,3973,11.99,2007-04-06 21:26:57.996577
7,29136,13,2,8831,11.99,2007-04-29 21:06:07.996577


**Zadanie 7**

Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.


In [6]:
country_ids = pd.read_sql("select country_id from country where country in ('Poland', 'Nigeria', 'Bangladesh')",con=connection)['country_id'].tolist()
city_ids = pd.read_sql(f"select city_id from city where country_id in ({','.join(str(c) for c in country_ids)})",con=connection)['city_id'].tolist()
address_ids = pd.read_sql(f"select address_id from address where city_id in ({','.join(str(c) for c in city_ids)})", con=connection)['address_id'].tolist()
PolNigBang_customers = pd.read_sql(f"select * from customer where address_id in ({','.join(str(a) for a in address_ids)})", con=connection)
PolNigBang_customers

,customer_id,store_id,first_name,last_name,email,address_id,activebool,create_date,last_update,active
0,18,2,Carol,Garcia,carol.garcia@sakilacustomer.org,22,True,2006-02-14,2013-05-26 14:49:45.738,1
1,21,1,Michelle,Clark,michelle.clark@sakilacustomer.org,25,True,2006-02-14,2013-05-26 14:49:45.738,1
2,80,1,Marilyn,Ross,marilyn.ross@sakilacustomer.org,84,True,2006-02-14,2013-05-26 14:49:45.738,1
3,103,1,Gladys,Hamilton,gladys.hamilton@sakilacustomer.org,107,True,2006-02-14,2013-05-26 14:49:45.738,1
4,128,1,Marjorie,Tucker,marjorie.tucker@sakilacustomer.org,132,True,2006-02-14,2013-05-26 14:49:45.738,1
5,156,1,Bertha,Ferguson,bertha.ferguson@sakilacustomer.org,160,True,2006-02-14,2013-05-26 14:49:45.738,1
6,198,2,Elsie,Kelley,elsie.kelley@sakilacustomer.org,202,True,2006-02-14,2013-05-26 14:49:45.738,1
7,232,2,Constance,Reid,constance.reid@sakilacustomer.org,236,True,2006-02-14,2013-05-26 14:49:45.738,1
8,250,2,Jo,Fowler,jo.fowler@sakilacustomer.org,254,True,2006-02-14,2013-05-26 14:49:45.738,1
9,270,1,Leah,Curtis,leah.curtis@sakilacustomer.org,275,True,2006-02-14,2013-05-26 14:49:45.738,1


**Zadanie 8**

Gdzie mieszkają członkowie personelu?


In [7]:
address_ids = pd.read_sql("select address_id from staff", con=connection)["address_id"].tolist()
staff_addresses = pd.read_sql(f"select * from address where address_id in ({','.join(str(a) for a in address_ids)})", con=connection)
staff_addresses

,address_id,address,address2,district,city_id,postal_code,phone,last_update
0,3,23 Workhaven Lane,None,Alberta,300,,14033335568,2006-02-15 09:45:30
1,4,1411 Lillydale Drive,None,QLD,576,,6172235589,2006-02-15 09:45:30


**Zadanie 9**

Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [8]:
staff_city_ids = staff_addresses['city_id'].tolist()
staff_country_ids = pd.read_sql(f"select country_id from city where city_id in ({','.join(str(c) for c in staff_city_ids)})", con=connection)['country_id'].tolist()
staff_countries = pd.read_sql(f"select country from country where country_id in ({','.join(str(c) for c in staff_country_ids)})", con=connection)
staff_countries

,country
0,Australia
1,Canada


**Zadanie 10**

Jakie kategorie filmów zostały wypożyczone przez klientów?


In [9]:
df = pd.read_sql('select name from category where category_id in (select category_id from film_category where film_id in (select film_id from film where film_id in (select film_id from inventory where inventory_id in (select inventory_id from rental)))) ', con=connection)
df

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


**Zadanie 11**

Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [10]:
df = pd.read_sql("select name from category where category_id in (select category_id from film_category where film_id in (select film_id from film where film_id in (select film_id from inventory where inventory_id in (select inventory_id from rental where customer_id in (select customer_id from customer where address_id in (select address_id from address where city_id in (select city_id from city where country_id in (select country_id from country where country = 'United States')))))))) ", con=connection)
df

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


**Zadanie 12**

Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [11]:
df = pd.read_sql("select title from film where film_id in (select film_id from film_actor where actor_id in (select actor_id from actor where (first_name = 'Olympia' AND last_name = 'Pfeiffer') OR (first_name = 'Julia' AND last_name = 'Zellweger') OR (first_name = 'Ellen' AND last_name = 'Presley')))", con=connection)
df

,title
0,Express Lonely
1,Bilko Anonymous
2,Effect Gladiator
3,Contact Anonymous
4,Others Soup
...,...
64,Hanky October
65,Tourist Pelican
66,Detective Vision
67,None Spiking
